In [1]:
from datasets import load_dataset, Dataset, Features, Image, Value
from pathlib import Path
import json

from huggingface_hub import HfApi

/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd dataset/

/Users/bitan/Desktop/medgemma/dataset


/Users/bitan/Desktop/medgemma/venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
records = []
for line in Path("dataset.jsonl").read_text().splitlines():
    item = json.loads(line)
    records.append({
        "image": item["image"],                     # provided image
        "tooth": item["tooth"],
        "type": item["type"],
        "treatment": item["treatment"],
        "diagnosis": item["diagnosis"],
        "bbox": item["bbox"],
        "text_summary": item["tooth"],
        "json_summary": item["tooth"]
    })

# Define features explicitly
features = Features({
    "image": Image(mode="RGB"),
    "tooth": Value("string"),
    "type": Value("string"),
    "treatment": Value("string"),
    "diagnosis": Value("string"),
    "bbox": [Value("int32")],
    "text_summary": Value("string"),
    "json_summary": Value("string"),
})


In [12]:
ds = Dataset.from_list(records, features=features)

In [13]:
print(ds.features)

{'image': Image(mode='RGB', decode=True), 'tooth': Value('string'), 'type': Value('string'), 'treatment': Value('string'), 'diagnosis': Value('string'), 'bbox': List(Value('int32')), 'text_summary': Value('string'), 'json_summary': Value('string')}


In [14]:
ds.push_to_hub(
    repo_id="justacoderwhocodes/dental_panoramic_closeup_dataset",
    split="train",
    private=True,
    commit_message="Trying something else"
)


Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 10.93ba/s]
Processing Files (1 / 1): 100%|██████████| 71.7MB / 71.7MB, 25.0kB/s  
New Data Upload: 100%|██████████| 65.1kB / 65.1kB, 25.0kB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.61s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/justacoderwhocodes/dental_panoramic_closeup_dataset/commit/f4a3d2a20e6bd76983d41f08d95fe9af56320472', commit_message='Trying something else', commit_description='', oid='f4a3d2a20e6bd76983d41f08d95fe9af56320472', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/justacoderwhocodes/dental_panoramic_closeup_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='justacoderwhocodes/dental_panoramic_closeup_dataset'), pr_revision=None, pr_num=None)

In [15]:
check_ds = load_dataset("justacoderwhocodes/dental_panoramic_closeup_dataset", split="train")

Generating train split: 100%|██████████| 1071/1071 [00:00<00:00, 13445.13 examples/s]


In [16]:
check_ds.features

{'image': Image(mode='RGB', decode=True),
 'tooth': Value('string'),
 'type': Value('string'),
 'treatment': Value('string'),
 'diagnosis': Value('string'),
 'bbox': List(Value('int32')),
 'text_summary': Value('string'),
 'json_summary': Value('string')}